In [2]:
import io
import os
import re
import nltk 
import spacy
import pandas as pd
import docx2txt
import constants as cs
import string
import re
import os
import utils
import spacy
import pprint
from spacy.matcher import Matcher
import multiprocessing as mp 
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import re
import nltk
from nltk.corpus import stopwords

In [3]:
import PyPDF2 as pdf
file = open('1.pdf', 'rb')
file

<_io.BufferedReader name='1.pdf'>

In [4]:
pdf_reader = pdf.PdfFileReader(file)
pdf_reader

In [55]:
pdf_reader.getNumPages()
page1 = pdf_reader.getPage(0)
resumeText = page1.extractText()[:5000]
resumeText      

'  Name  : ROBERT  \n \n9475838374  \nEmail:  info@qwikresume.com  Website:  www.qwikresume.com  \nLinkedIn:  linkedin.com/qwikresume  Address:  1737 Marshville  Road,  \nAlabama.  \nObjective   \nOver 6 years  of IT industry  experience  with 4+ years  of experience  as Mobile  application  \ndeveloper in the field of Android. Experience in developing front end applications for Android  \nphones.  Experience  developing  mobile  applications  on Android  platform,  building  Custom  UI using \nViews,  ViewGroups,  Layouts,  Widgets  and graphics  that scale  based  on the screen  size using  9- \npatch  images,  localization, testing  and publishing  the applications  to the Android  Market.  \n \nSkills   \nPython,  Java,  C, Javascript,  Matlab,  R. \n \nWork  Experience   \nAndroid  Developer  \nABC Corporation  \xad January  2011  – March  2012  \n• Environment  Eclipse  IDE, Android  Studio,  GenyMotion,  Java,  Android  SDK, Android  \nDevelopment Tools (ADT), JSON, XML, \n• Inv

In [30]:
def cleanResume(resumeText):
    #resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
   # resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    #resumeText = re.sub(r'\n', ',', resumeText)
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
   # resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
   # resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
   # resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*,-/<=>?[]^`{|}~"""), ' ', resumeText)
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText
clean = cleanResume(resumeText)
clean

' Name : ROBERT 9475838374 Email: info@qwikresume.com Website: www.qwikresume.com LinkedIn: linkedin.com/qwikresume Address: 1737 Marshville Road, Alabama. Objective Over 6 years of IT industry experience with 4+ years of experience as Mobile application developer in the field of Android. Experience in developing front end applications for Android phones. Experience developing mobile applications on Android platform, building Custom UI using Views, ViewGroups, Layouts, Widgets and graphics that scale based on the screen size using 9- patch images, localization, testing and publishing the applications to the Android Market. Skills Python, Java, C, Javascript, Matlab, R. Work Experience Android Developer ABC Corporation January 2011 March 2012 Environment Eclipse IDE, Android Studio, GenyMotion, Java, Android SDK, Android Development Tools (ADT), JSON, XML, Involved in the full life cycle of the project including analysis design, '

In [31]:
" ".join(clean.split())

'Name : ROBERT 9475838374 Email: info@qwikresume.com Website: www.qwikresume.com LinkedIn: linkedin.com/qwikresume Address: 1737 Marshville Road, Alabama. Objective Over 6 years of IT industry experience with 4+ years of experience as Mobile application developer in the field of Android. Experience in developing front end applications for Android phones. Experience developing mobile applications on Android platform, building Custom UI using Views, ViewGroups, Layouts, Widgets and graphics that scale based on the screen size using 9- patch images, localization, testing and publishing the applications to the Android Market. Skills Python, Java, C, Javascript, Matlab, R. Work Experience Android Developer ABC Corporation January 2011 March 2012 Environment Eclipse IDE, Android Studio, GenyMotion, Java, Android SDK, Android Development Tools (ADT), JSON, XML, Involved in the full life cycle of the project including analysis design,'

# Trail tokenize and extract skills

In [32]:
from nltk.tokenize import word_tokenize
tokens = word_tokenize(clean)
print(tokens)

['Name', ':', 'ROBERT', '9475838374', 'Email', ':', 'info', '@', 'qwikresume.com', 'Website', ':', 'www.qwikresume.com', 'LinkedIn', ':', 'linkedin.com/qwikresume', 'Address', ':', '1737', 'Marshville', 'Road', ',', 'Alabama', '.', 'Objective', 'Over', '6', 'years', 'of', 'IT', 'industry', 'experience', 'with', '4+', 'years', 'of', 'experience', 'as', 'Mobile', 'application', 'developer', 'in', 'the', 'field', 'of', 'Android', '.', 'Experience', 'in', 'developing', 'front', 'end', 'applications', 'for', 'Android', 'phones', '.', 'Experience', 'developing', 'mobile', 'applications', 'on', 'Android', 'platform', ',', 'building', 'Custom', 'UI', 'using', 'Views', ',', 'ViewGroups', ',', 'Layouts', ',', 'Widgets', 'and', 'graphics', 'that', 'scale', 'based', 'on', 'the', 'screen', 'size', 'using', '9-', 'patch', 'images', ',', 'localization', ',', 'testing', 'and', 'publishing', 'the', 'applications', 'to', 'the', 'Android', 'Market', '.', 'Skills', 'Python', ',', 'Java', ',', 'C', ',', 'J

In [33]:
from nltk.tokenize import sent_tokenize
sent = sent_tokenize(clean)
print(sent)

[' Name : ROBERT 9475838374 Email: info@qwikresume.com Website: www.qwikresume.com LinkedIn: linkedin.com/qwikresume Address: 1737 Marshville Road, Alabama.', 'Objective Over 6 years of IT industry experience with 4+ years of experience as Mobile application developer in the field of Android.', 'Experience in developing front end applications for Android phones.', 'Experience developing mobile applications on Android platform, building Custom UI using Views, ViewGroups, Layouts, Widgets and graphics that scale based on the screen size using 9- patch images, localization, testing and publishing the applications to the Android Market.', 'Skills Python, Java, C, Javascript, Matlab, R. Work Experience Android Developer ABC Corporation January 2011 March 2012 Environment Eclipse IDE, Android Studio, GenyMotion, Java, Android SDK, Android Development Tools (ADT), JSON, XML, Involved in the full life cycle of the project including analysis design,']


# Convert Lowercase

In [62]:
lis = []

for word in tokens:
    lis.append(word.lower())
print(lis)


['name', ':', 'robert', '9475838374', 'email', ':', 'info', '@', 'qwikresume.com', 'website', ':', 'www.qwikresume.com', 'linkedin', ':', 'linkedin.com/qwikresume', 'address', ':', '1737', 'marshville', 'road', ',', 'alabama', '.', 'objective', 'over', '6', 'years', 'of', 'it', 'industry', 'experience', 'with', '4+', 'years', 'of', 'experience', 'as', 'mobile', 'application', 'developer', 'in', 'the', 'field', 'of', 'android', '.', 'experience', 'in', 'developing', 'front', 'end', 'applications', 'for', 'android', 'phones', '.', 'experience', 'developing', 'mobile', 'applications', 'on', 'android', 'platform', ',', 'building', 'custom', 'ui', 'using', 'views', ',', 'viewgroups', ',', 'layouts', ',', 'widgets', 'and', 'graphics', 'that', 'scale', 'based', 'on', 'the', 'screen', 'size', 'using', '9-', 'patch', 'images', ',', 'localization', ',', 'testing', 'and', 'publishing', 'the', 'applications', 'to', 'the', 'android', 'market', '.', 'skills', 'python', ',', 'java', ',', 'c', ',', 'j

In [43]:
str = lis

str1 = [

   'python',
    'c',
    'javascript',
    'matlab'
    
]

a = set(str)
b = set(str1)
c = []

for i in a:
    if i in b:
        c.append(i)
print("Name",c)

Name ['python', 'matlab', 'c', 'javascript']


# Lemmitization

In [34]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to C:\Users\BALAJI
[nltk_data]     K\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [35]:
def lemmatize_text(text):
    w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
    lemmatize_text(line)
lemman = lemmatize_text(clean)
print(lemman)

['Name', ':', 'ROBERT', '9475838374', 'Email:', 'info@qwikresume.com', 'Website:', 'www.qwikresume.com', 'LinkedIn:', 'linkedin.com/qwikresume', 'Address:', '1737', 'Marshville', 'Road,', 'Alabama.', 'Objective', 'Over', '6', 'year', 'of', 'IT', 'industry', 'experience', 'with', '4+', 'year', 'of', 'experience', 'a', 'Mobile', 'application', 'developer', 'in', 'the', 'field', 'of', 'Android.', 'Experience', 'in', 'developing', 'front', 'end', 'application', 'for', 'Android', 'phones.', 'Experience', 'developing', 'mobile', 'application', 'on', 'Android', 'platform,', 'building', 'Custom', 'UI', 'using', 'Views,', 'ViewGroups,', 'Layouts,', 'Widgets', 'and', 'graphic', 'that', 'scale', 'based', 'on', 'the', 'screen', 'size', 'using', '9-', 'patch', 'images,', 'localization,', 'testing', 'and', 'publishing', 'the', 'application', 'to', 'the', 'Android', 'Market.', 'Skills', 'Python,', 'Java,', 'C,', 'Javascript,', 'Matlab,', 'R.', 'Work', 'Experience', 'Android', 'Developer', 'ABC', 'Cor

# After lemmatize for tokenize

In [36]:
from nltk.tokenize import word_tokenize
tokens = word_tokenize(lemman)
print(tokens)

TypeError: expected string or bytes-like object

### Trial extracting skills using Lemmintization

In [18]:
str = lemman

str1 = [
    "Python,",
    'C',
    'Matlab',
]

a = set(str)
b = set(str1)
c = []

for i in a:
    if i in b:
        c.append(i)
print("Skills",c)


Skills ['Python,']


# trail
resumeText = nltk.word_tokenize(clean)
resumeText

In [8]:
def extract_entity_sections(text):
    
    RESUME_SECTIONS_GRAD = [
                        'accomplishments',
                        'experience',
                        'education',
                        'interests',
                        'projects',
                        'professional experience',
                        'publications',
                        'skills',
                        'certifications',
                        'objective',
                        'career objective',
                        'summary',
                        'leadership'
                    ]

    text_split = [i.strip() for i in text.split('\n')]
    entities = {}
    key = False
    for phrase in text_split:
        if len(phrase) == 1:
            p_key = phrase
            print(p_key)
        else:
            p_key = set(phrase.lower().split())&set(RESUME_SECTIONS_GRAD)
            print(p_key)
        try:
            p_key = list(p_key)[0]
            print(p_key)
        except IndexError:
            pass
        if p_key in RESUME_SECTIONS_GRAD:
            entities[p_key] = []
            key = p_key
        elif key and phrase.strip():
            entities[key].append(phrase)

Skillset = extract_entity_sections(clean)
print(Skillset)


{'experience', 'objective', 'skills'}
experience
None


In [9]:
from pdfminer.high_level import extract_text
 
EMAIL_REG = re.compile(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+')
def extract_emails(text):
    return re.findall(EMAIL_REG, clean)
text = clean
emails = extract_emails(text)
 
if emails:
    mail = emails[0]
    print(mail)

info@qwikresume.com


In [10]:
def extract_email(text):
    
    email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)",str(text))
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None
print(extract_email(clean))

info@qwikresume.com


# trail


import spacy
import en_core_web_sm
from spacy.matcher import Matcher

# load pre-trained model
nlp = en_core_web_sm.load()

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)

NAME_PATTERN = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]

def extract_name(text,matcher):
    email = extract_email(text)
    email = email.rsplit('@',1)[0]
    email = re.sub(r'([^a-zA-Z]+?)',' ',email)
    
    f = string.capwords(email[0])
    l = string.capwords(email[-1])
    pattern = [NAME_PATTERN]
    matcher.add('NAME', [pattern], on_match=None)
    matches = matcher(text)
    for match_id,start,end in matches:
        span = text[start:end]
        if f in span.text:
            return span.text
        elif l in span.text:
            return span.text
        elif f.upper() in span.text:
            return span.text
        elif l.upper() in span.text:
            return span.text
        
extract_name(resumeText,matcher)
        
    

# trail 

In [44]:
import spacy
import en_core_web_sm
from spacy.matcher import Matcher

# load pre-trained model
nlp = en_core_web_sm.load()

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)


def extract_name(resume_text):
    nlp_text = nlp(resume_text)
    
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', [pattern], on_match=None)
    
    matches = matcher(nlp_text)
    
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text
name = extract_name(clean)
print('Name: ',name)

Name:  Marshville Road


In [12]:
import spacy
from spacy.matcher import Matcher

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)

def extract_name(resume_text):
    nlp_text = nlp(resume_text)
    
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', [pattern], on_match=None)
    
    matches = matcher(nlp_text)
    
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text
    
extract_name(clean)

'Marshville Road'

In [13]:
def extract_mobile_number(resume_text):
    phone = re.findall(re.compile(r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'), resume_text)
    
    if phone:
        number = ''.join(phone[0])
        if len(number) > 10:
            return number
        else:
            return number
mobile = extract_mobile_number(clean)
print('Mobile Number: ',mobile)

Mobile Number:  9475838374


# trail

In [35]:
def skills(nlp_text,noun_chunks):
    tokens = [tokens.text for token in nlp_text if not token.is_stop]
    data = pd.read_csv(os.path.join(os.path.dirname(__file__),'skills.csv'))
    skills = list(data.columns.values)
    skillset = []
    
    #check for one-grams
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    #check bigram trigram
        for token in noun_chunks:
        token = token.text.lower().strip()
        if token in skills:
            skillset.append(token)
    return [i.capitalize() for i in set([i.lower() for i in skillset])]

NameError: free variable 'tokens' referenced before assignment in enclosing scope

In [33]:
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_sm")  # load the English model
doc = nlp(clean)    # process a text and create a Doc object
for chunk in doc.noun_chunks:       # iterate over the noun chunks in the Doc

    noun_chunks = chunk.text

def extract_skills(resume_text):
    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    colnames = ['Skills']
    # reading the csv file
    data =   [
    'machine learning',
    'data science',
    'Java',
    'python',
    'word',
    'excel',
    'English',
    'Java',
    'C',
]
 
    
    # extract values
    skills = data.Skills.tolist()
    
    skillset = []
    
    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
   
    for token in noun_chunks:
        token = token.text.lower().strip()
        if token in skills:
            skillset.append(token)
    return [i.capitalize() for i in set([i.lower() for i in skillset])]
  
print ('Skills',extract_skills(clean))

AttributeError: 'list' object has no attribute 'Skills'

# Take Domain

In [ ]:


###Edites
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_sm")  # load the English model
doc = nlp(clean)    # process a text and create a Doc object
for chunk in doc.noun_chunks:       # iterate over the noun chunks in the Doc

    noun_chunks = chunk.text

def extract_skills(resume_text):
    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    # reading the csv file
    data =   [
    'machine learning',
    'data science',
    'Java',
    'python',
    'word',
    'excel',
    'English',
    'Java',
    'C',
    'JavaScript'
]
 
    
    # extract values
    skills = data
    
    skillset = []
    
    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
   
        if noun_chunks in skills:
            skillset.append(noun_chunks)
    return [i.capitalize() for i in set([i.lower() for i in skillset])]
  
print ('Skills',extract_skills(clean))

In [126]:
nlp = spacy.load("en_core_web_sm")  # load the English model
doc = nlp(clean)    # process a text and create a Doc object
for chunk in doc.noun_chunks:       # iterate over the noun chunks in the Doc
   print(chunk.text)

noun_chunks = chunk.text

SM ITH Phone
Email info
Website www qwikresume
com LinkedIn
com qwikresume
Address 1737 Marshville Road Alabama Objective
Over 6 years
IT industry experience
4 years
experience
Mobile application developer
the field
Android Experience
front end applications
Android phones
mobile applications
Android platform building Custom UI
Views ViewGroups Layouts Widg
the screen size
9 patch images localization testing
the applications
the Android Market Skills Python
Java C Javascript Matlab R Work Experience
Android Developer ABC Corporation
Environment Eclipse IDE Android Studio GenyMotion
Java Android SDK Android Development Tools ADT JSON XML
the full life cycle
the project


# Take Domain1

In [15]:
import docx2txt
import nltk
 
nltk.download('stopwords')
 
# you may read the database from a csv file or some other database
SKILLS_DB = [
    'machine learning',
    'data science',
    'Java',
    'python',
    'word',
    'excel',
    'English',
    'Java',
    'C',
]
 
 
from pdfminer.high_level import extract_text
 
 
def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)
 
 
 
def extract_skills(input_text):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = nltk.tokenize.word_tokenize(input_text)
 
    # remove the stop words
    filtered_tokens = [w for w in word_tokens if w not in stop_words]
 
    # remove the punctuation
    filtered_tokens = [w for w in word_tokens if w.isalpha()]
 
    # generate bigrams and trigrams (such as artificial intelligence)
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))
 
    # we create a set to keep the results in.
    found_skills = set()
 
    # we search for each token in our skills database
    for token in filtered_tokens:
        if token.lower() in SKILLS_DB:
            found_skills.add(token)
 
    # we search for each bigram and trigram in our skills database
    for ngram in bigrams_trigrams:
        if ngram.lower() in SKILLS_DB:
            found_skills.add(ngram)
 
    return found_skills
 
 
if __name__ == '__main__':
    text = extract_text_from_pdf('android-developer-1559034496.pdf')
    skills = extract_skills(text)
 
    print(skills)  # noqa: T001

[nltk_data] Downloading package stopwords to C:\Users\BALAJI
[nltk_data]     K\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'Python'}


In [16]:
def extract_address(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ in['LOC','GPE']:
            return ent.text
        
print(extract_address(clean))

Mobile


In [65]:
import spacy
from spacy import displacy

NER = spacy.load("en_core_web_sm")

In [66]:
text1= NER(clean)

In [76]:
for word in text1.ents:
    print(word.text,word.label_)

ROBERT SM ORG
9475838374 CARDINAL
LinkedIn PERSON
1737 DATE
Marshville Road, Alabama FAC
6 years DATE
4+ years DATE
Mobile GPE
Android ORG
Android ORG
Android ORG
Custom UI ORG
Views ORG
ViewGroups ORG
Layouts NORP
Widgets ORG
the Android Market ORG
Skills Python ORG
Matlab PERSON
R. NORP
January 2011 March 2012 DATE
Android Studio PERSON
GenyMotion ORG
Java PERSON
SDK ORG
Android Development Tools ORG
ADT ORG


In [69]:
spacy.explain("PRODUCT")

'Objects, vehicles, foods, etc. (not services)'

In [17]:
def extract_address(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ in['LOC','GPE']:
            return ent.text
print(extract_address(clean))

Mobile


In [96]:
 print( 
            "email"+mail,"phone"+ mobile,"name"+name,"skills"+skills,
            "Skillset"+skillset,
 )

SyntaxError: invalid syntax (<ipython-input-96-b7b0bc5c0b07>, line 2)

# Convert lower case 

In [59]:
lang = ["A# .NET","A# (Axiom)","A-0 System","A+","A++","ABAP","ABC","ABC ALGOL","ABLE","ABSET","ABSYS","ACC","Accent","Ace DASL","ACL2","ACT-III","Action!","ActionScript","Ada","Adenine","Agda","Agilent VEE","Agora","AIMMS","Alef","ALF","ALGOL 58","ALGOL 60","ALGOL 68","ALGOL W","Alice","Alma-0","AmbientTalk","Amiga E","AMOS","AMPL","APL","App Inventor for Android's visual block language","AppleScript","Arc","ARexx","Argus","AspectJ","Assembly language","ATS","Ateji PX","AutoHotkey","Autocoder","AutoIt","AutoLISP / Visual LISP","Averest","AWK","Axum","B","Babbage","Bash","BASIC","bc","BCPL","BeanShell","Batch (Windows/Dos)","Bertrand","BETA","Bigwig","Bistro","BitC","BLISS","Blue","Bon","Boo","Boomerang","Bourne shell","bash","ksh","BREW","BPEL","C","C--","C++","C#","C/AL","Caché ObjectScript","C Shell","Caml","Candle","Cayenne","CDuce","Cecil","Cel","Cesil","Ceylon","CFEngine","CFML","Cg","Ch","Chapel","CHAIN","Charity","Charm","Chef","CHILL","CHIP-8","chomski","ChucK","CICS","Cilk","CL","Claire","Clarion","Clean","Clipper","CLIST","Clojure","CLU","CMS-2","COBOL","Cobra","CODE","CoffeeScript","Cola","ColdC","ColdFusion","COMAL","Combined Programming Language","COMIT","Common Intermediate Language","Common Lisp","COMPASS","Component Pascal","Constraint Handling Rules","Converge","Cool","Coq","Coral 66","Corn","CorVision","COWSEL","CPL","csh","CSP","Csound","CUDA","Curl","Curry","Cyclone","Cython","D","DASL","DASL","Dart","DataFlex","Datalog","DATATRIEVE","dBase","dc","DCL","Deesel","Delphi","DinkC","DIBOL","Dog","Draco","DRAKON","Dylan","DYNAMO","E","E#","Ease","Easy PL/I","Easy Programming Language","EASYTRIEVE PLUS","ECMAScript","Edinburgh IMP","EGL","Eiffel","ELAN","Elixir","Elm","Emacs Lisp","Emerald","Epigram","EPL","Erlang","es","Escapade","Escher","ESPOL","Esterel","Etoys","Euclid","Euler","Euphoria","EusLisp Robot Programming Language","CMS EXEC","EXEC 2","Executable UML","F","F#","Factor","Falcon","Fancy","Fantom","FAUST","Felix","Ferite","FFP","Fjölnir","FL","Flavors","Flex","FLOW-MATIC","FOCAL","FOCUS","FOIL","FORMAC","@Formula","Forth","Fortran","Fortress","FoxBase","FoxPro","FP","FPr","Franz Lisp","Frege","F-Script","FSProg","G","Google Apps Script","Game Maker Language","GameMonkey Script","GAMS","GAP","G-code","Genie","GDL","Gibiane","GJ","GEORGE","GLSL","GNU E","GM","Go","Go!","GOAL","Gödel","Godiva","GOM (Good Old Mad)","Goo","Gosu","GOTRAN","GPSS","GraphTalk","GRASS","Groovy","Hack (programming language)","HAL/S","Hamilton C shell","Harbour","Hartmann pipelines","Haskell","Haxe","High Level Assembly","HLSL","Hop","Hope","Hugo","Hume","HyperTalk","IBM Basic assembly language","IBM HAScript","IBM Informix-4GL","IBM RPG","ICI","Icon","Id","IDL","Idris","IMP","Inform","Io","Ioke","IPL","IPTSCRAE","ISLISP","ISPF","ISWIM","J","J#","J++","JADE","Jako","JAL","Janus","JASS","Java","JavaScript","JCL","JEAN","Join Java","JOSS","Joule","JOVIAL","Joy","JScript","JScript .NET","JavaFX Script","Julia","Jython","K","Kaleidoscope","Karel","Karel++","KEE","Kixtart","KIF","Kojo","Kotlin","KRC","KRL","KUKA","KRYPTON","ksh","L","L# .NET","LabVIEW","Ladder","Lagoona","LANSA","Lasso","LaTeX","Lava","LC-3","Leda","Legoscript","LIL","LilyPond","Limbo","Limnor","LINC","Lingo","Linoleum","LIS","LISA","Lisaac","Lisp","Lite-C","Lithe","Little b","Logo","Logtalk","LPC","LSE","LSL","LiveCode","LiveScript","Lua","Lucid","Lustre","LYaPAS","Lynx","M2001","M4","Machine code","MAD","MAD/I","Magik","Magma","make","Maple","MAPPER","MARK-IV","Mary","MASM Microsoft Assembly x86","Mathematica","MATLAB","Maxima","Macsyma","Max","MaxScript","Maya (MEL)","MDL","Mercury","Mesa","Metacard","Metafont","MetaL","Microcode","MicroScript","MIIS","MillScript","MIMIC","Mirah","Miranda","MIVA Script","ML","Moby","Model 204","Modelica","Modula","Modula-2","Modula-3","Mohol","MOO","Mortran","Mouse","MPD","CIL","MSL","MUMPS","NASM","NATURAL","Napier88","Neko","Nemerle","nesC","NESL","Net.Data","NetLogo","NetRexx","NewLISP","NEWP","Newspeak","NewtonScript","NGL","Nial","Nice","Nickle","Nim","NPL","Not eXactly C","Not Quite C","NSIS","Nu","NWScript","NXT-G","o:XML","Oak","Oberon","Obix","OBJ2","Object Lisp","ObjectLOGO","Object REXX","Object Pascal","Objective-C","Objective-J","Obliq","Obol","OCaml","occam","occam-π","Octave","OmniMark","Onyx","Opa","Opal","OpenCL","OpenEdge ABL","OPL","OPS5","OptimJ","Orc","ORCA/Modula-2","Oriel","Orwell","Oxygene","Oz","P#","ParaSail (programming language)","PARI/GP","Pascal","Pawn","PCASTL","PCF","PEARL","PeopleCode","Perl","PDL","PHP","Phrogram","Pico","Picolisp","Pict","Pike","PIKT","PILOT","Pipelines","Pizza","PL-11","PL/0","PL/B","PL/C","PL/I","PL/M","PL/P","PL/SQL","PL360","PLANC","Plankalkül","Planner","PLEX","PLEXIL","Plus","POP-11","PostScript","PortablE","Powerhouse","PowerBuilder","PowerShell","PPL","Processing","Processing.js","Prograph","PROIV","Prolog","PROMAL","Promela","PROSE modeling language","PROTEL","ProvideX","Pro*C","Pure","Python","Q (equational programming language)","Q (programming language from Kx Systems)","Qalb","QtScript","QuakeC","QPL","R","R++","Racket","RAPID","Rapira","Ratfiv","Ratfor","rc","REBOL","Red","Redcode","REFAL","Reia","Revolution","rex","REXX","Rlab","RobotC","ROOP","RPG","RPL","RSL","RTL/2","Ruby","RuneScript","Rust","S","S2","S3","S-Lang","S-PLUS","SA-C","SabreTalk","SAIL","SALSA","SAM76","SAS","SASL","Sather","Sawzall","SBL","Scala","Scheme","Scilab","Scratch","Script.NET","Sed","Seed7","Self","SenseTalk","SequenceL","SETL","Shift Script","SIMPOL","SIGNAL","SiMPLE","SIMSCRIPT","Simula","Simulink","SISAL","SLIP","SMALL","Smalltalk","Small Basic","SML","Snap!","SNOBOL","SPITBOL","Snowball","SOL","Span","SPARK","Speedcode","SPIN","SP/k","SPS","Squeak","Squirrel","SR","S/SL","Stackless Python","Starlogo","Strand","Stata","Stateflow","Subtext","SuperCollider","SuperTalk","Swift (Apple programming language)","Swift (parallel scripting language)","SYMPL","SyncCharts","SystemVerilog","T","TACL","TACPOL","TADS","TAL","Tcl","Tea","TECO","TELCOMP","TeX","TEX","TIE","Timber","TMG","Tom","TOM","Topspeed","TPU","Trac","TTM","T-SQL","TTCN","Turing","TUTOR","TXL","TypeScript","Turbo C++","Ubercode","UCSD Pascal","Umple","Unicon","Uniface","UNITY","Unix shell","UnrealScript","Vala","VBA","VBScript","Verilog","VHDL","Visual Basic","Visual Basic .NET","Visual DataFlex","Visual DialogScript","Visual Fortran","Visual FoxPro","Visual J++","Visual J#","Visual Objects","Visual Prolog","VSXu","Vvvv","WATFIV, WATFOR","WebDNA","WebQL","Windows PowerShell","Winbatch","Wolfram","Wyvern","X++","X#","X10","XBL","XC","XMOS architecture","xHarbour","XL","Xojo","XOTcl","XPL","XPL0","XQuery","XSB","XSLT","XPath","Xtend","Yorick","YQL","Z notation","Zeno","ZOPL","ZPL"]
lis = []

for word in lang:
    lis.append(word.lower())
print(lis)

['a# .net', 'a# (axiom)', 'a-0 system', 'a+', 'a++', 'abap', 'abc', 'abc algol', 'able', 'abset', 'absys', 'acc', 'accent', 'ace dasl', 'acl2', 'act-iii', 'action!', 'actionscript', 'ada', 'adenine', 'agda', 'agilent vee', 'agora', 'aimms', 'alef', 'alf', 'algol 58', 'algol 60', 'algol 68', 'algol w', 'alice', 'alma-0', 'ambienttalk', 'amiga e', 'amos', 'ampl', 'apl', "app inventor for android's visual block language", 'applescript', 'arc', 'arexx', 'argus', 'aspectj', 'assembly language', 'ats', 'ateji px', 'autohotkey', 'autocoder', 'autoit', 'autolisp / visual lisp', 'averest', 'awk', 'axum', 'b', 'babbage', 'bash', 'basic', 'bc', 'bcpl', 'beanshell', 'batch (windows/dos)', 'bertrand', 'beta', 'bigwig', 'bistro', 'bitc', 'bliss', 'blue', 'bon', 'boo', 'boomerang', 'bourne shell', 'bash', 'ksh', 'brew', 'bpel', 'c', 'c--', 'c++', 'c#', 'c/al', 'caché objectscript', 'c shell', 'caml', 'candle', 'cayenne', 'cduce', 'cecil', 'cel', 'cesil', 'ceylon', 'cfengine', 'cfml', 'cg', 'ch', 'cha

In [61]:
lis = []
for word in clean:
    lis.append(word.lower())
print(lis)

[' ', 'n', 'a', 'm', 'e', ' ', ':', ' ', 'r', 'o', 'b', 'e', 'r', 't', ' ', '9', '4', '7', '5', '8', '3', '8', '3', '7', '4', ' ', 'e', 'm', 'a', 'i', 'l', ':', ' ', 'i', 'n', 'f', 'o', '@', 'q', 'w', 'i', 'k', 'r', 'e', 's', 'u', 'm', 'e', '.', 'c', 'o', 'm', ' ', 'w', 'e', 'b', 's', 'i', 't', 'e', ':', ' ', 'w', 'w', 'w', '.', 'q', 'w', 'i', 'k', 'r', 'e', 's', 'u', 'm', 'e', '.', 'c', 'o', 'm', ' ', 'l', 'i', 'n', 'k', 'e', 'd', 'i', 'n', ':', ' ', 'l', 'i', 'n', 'k', 'e', 'd', 'i', 'n', '.', 'c', 'o', 'm', '/', 'q', 'w', 'i', 'k', 'r', 'e', 's', 'u', 'm', 'e', ' ', 'a', 'd', 'd', 'r', 'e', 's', 's', ':', ' ', '1', '7', '3', '7', ' ', 'm', 'a', 'r', 's', 'h', 'v', 'i', 'l', 'l', 'e', ' ', 'r', 'o', 'a', 'd', ',', ' ', 'a', 'l', 'a', 'b', 'a', 'm', 'a', '.', ' ', 'o', 'b', 'j', 'e', 'c', 't', 'i', 'v', 'e', ' ', 'o', 'v', 'e', 'r', ' ', '6', ' ', 'y', 'e', 'a', 'r', 's', ' ', 'o', 'f', ' ', 'i', 't', ' ', 'i', 'n', 'd', 'u', 's', 't', 'r', 'y', ' ', 'e', 'x', 'p', 'e', 'r', 'i', 'e',